<a href="https://colab.research.google.com/github/ayoub-kplr/AI-Architecture-Cloud/blob/main/Azure/python%20Notebooks/09_Create_a_Real_time_Inferencing_Service.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Créer un service d'inférence en temps réel

Après avoir formé un modèle prédictif, vous pouvez le déployer en tant que service en temps réel que les clients peuvent utiliser pour obtenir des prédictions à partir de nouvelles données.


## Connectez-vous à votre espace de travail

Pour commencer, connectez-vous à votre espace de travail.

> **Remarque** : Si vous n'avez pas encore établi de session authentifiée avec votre abonnement Azure, vous serez invité à vous authentifier en cliquant sur un lien, en saisissant un code d'authentification et en vous connectant à Azure.


In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Former et enregistrer un modèle

Maintenant, formons et enregistrons un modèle.


In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name="mslearn-train-diabetes")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

## Déployer le modèle en tant que service Web

Vous avez formé et enregistré un modèle d'apprentissage automatique qui classe les patients en fonction de la probabilité qu'ils souffrent de diabète. Ce modèle pourrait être utilisé dans un environnement de production tel qu'un cabinet médical où seuls les patients jugés à risque doivent être soumis à un test clinique pour le diabète. Pour prendre en charge ce scénario, vous allez déployer le modèle en tant que service Web.

Tout d'abord, déterminons quels modèles vous avez enregistrés dans l'espace de travail.


In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

Bon, obtenons maintenant le modèle que nous voulons déployer. Par défaut, si nous spécifions un nom de modèle, la dernière version sera renvoyée.


In [ ]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

Nous allons créer un service Web pour héberger ce modèle, et cela nécessitera du code et des fichiers de configuration ; alors créons un dossier pour ceux-ci.


In [ ]:
import os

# Create a folder for the deployment files
deployment_folder = './diabetes_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder,script_file)

Le service Web sur lequel nous déployons le modèle aura besoin de code Python pour charger les données d'entrée, obtenir le modèle à partir de l'espace de travail, générer et renvoyer des prédictions. Nous enregistrerons ce code dans un *script d'entrée* (souvent appelé *script de notation*) qui sera déployé sur le service Web.

Le script se compose de deux fonctions :

- **init** : Cette fonction est appelée lors de l'initialisation du service, et est généralement utilisée pour charger le modèle. Notez que le script de notation utilise la variable d'environnement **AZUREML_MODEL_DIR** pour déterminer le dossier dans lequel le modèle est stocké.
- **run** : cette fonction est appelée chaque fois qu'une application cliente soumet de nouvelles données et est généralement utilisée pour inférer des prédictions à partir du modèle.


In [ ]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Le service Web sera hébergé dans un conteneur, et le conteneur devra installer toutes les dépendances Python requises lors de son initialisation. Dans ce cas, notre code d'évaluation nécessite **scikit-learn** et certains packages spécifiques à Azure Machine Learning qui sont utilisés par le service Web d'évaluation. Nous allons donc créer un environnement qui les inclut. Ensuite, nous ajouterons cet environnement à une *configuration d'inférence* avec le script de notation, et définirons une *configuration de déploiement* pour le conteneur dans lequel l'environnement et le script seront hébergés.

Nous pouvons ensuite déployer le modèle en tant que service basé sur ces configurations.

> **Plus d'informations** : pour plus de détails sur le déploiement du modèle et les options pour les environnements d'exécution cibles, consultez la [documentation](https://docs.microsoft.com/azure/machine-learning/how-to-deploy- et où).

Le déploiement prendra un certain temps car il exécute d'abord un processus pour créer une image de conteneur, puis exécute un processus pour créer un service Web basé sur l'image. Une fois le déploiement terminé avec succès, l'état **Sain** s'affiche.


In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
service_env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference")
service_env.inferencing_stack_version="latest"

inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model as a service
print('Deploying model...')
service_name = "diabetes-service"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)

J'espère que le déploiement a réussi et que vous pouvez voir un état **Sain**. Si ce n'est pas le cas, vous pouvez utiliser le code suivant pour obtenir les journaux de service afin de vous aider à résoudre les problèmes.


In [ ]:
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

Examinez votre espace de travail dans [Azure Machine Learning Studio](https://ml.azure.com) et affichez la page **Endpoints**, qui affiche les services déployés dans votre espace de travail.

Vous pouvez également récupérer les noms des services Web de votre espace de travail en exécutant le code suivant :


In [ ]:
for webservice_name in ws.webservices:
    print(webservice_name)

## Utiliser le service Web

Une fois le service déployé, vous pouvez désormais le consommer à partir d'une application cliente.


In [ ]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

Vous pouvez également envoyer plusieurs observations de patients au service et obtenir une prédiction pour chacune d'entre elles.


In [ ]:
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Le code ci-dessus utilise le SDK Azure Machine Learning pour se connecter au service Web conteneurisé et l'utiliser pour générer des prédictions à partir de votre modèle de classification du diabète. En production, un modèle est susceptible d'être consommé par des applications métier qui n'utilisent pas le SDK Azure Machine Learning, mais envoient simplement des requêtes HTTP au service web.

Déterminons l'URL à laquelle ces applications doivent soumettre leurs requêtes :


In [ ]:
endpoint = service.scoring_uri
print(endpoint)

Maintenant que vous connaissez l'URI du point de terminaison, une application peut simplement faire une requête HTTP, envoyer les données du patient au format JSON et recevoir en retour la ou les classes prédites.


In [ ]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Vous avez déployé votre service Web en tant que service Azure Container Instance (ACI) qui ne nécessite aucune authentification. Cela convient pour le développement et les tests, mais pour la production, vous devez envisager de déployer sur un cluster Azure Kubernetes Service (AKS) et d'activer l'authentification basée sur les jetons. Cela nécessiterait que les requêtes REST incluent un en-tête **Autorisation**.

## Supprimer le service

Lorsque vous n'avez plus besoin de votre service, vous devez le supprimer pour éviter d'encourir des frais inutiles.


In [ ]:
service.delete()
print ('Service deleted.')

Pour plus d'informations sur la publication d'un modèle en tant que service, consultez la [documentation](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)
